# *k*-Means Algorithm

The EM algorithm for *k*-means has the following procedure: 
1. Guess some initial cluster centres.
2. Repeat the following until convergence is reached:
    1. E-step: Assign points to the nearest cluster centre.
    2. M-step: Update the cluster centres with the mean position.

The E-step updates our expectation of which cluster each of the points belongs to, while the M-step maximises some fitness function that defines the cluster centre locations. 
Each repetition of the E-M loop will always result in a better estimate of the cluster characteristics. 

Let's try and write some code to perform this algorithm ourselves. 
First, we need some data to cluster, for this, we will use the `scikit-learn` method to produce random blobs of data. 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

X, y_true = make_blobs(n_samples=300, centers=4, cluster_std=1, random_state=0)
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(X[:, 0], X[:, 1], '.')
plt.show()

We will try and identify four clusters in this data, therefore, we set the following. 

In [2]:
n_clusters = 4

As a starting position, let's select four of the points randomly. 

In [3]:
import numpy as np

i = np.random.randint(0, X.shape[0], size=4)
centres = X[i]

Our initial guess is shown below with orange squares.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(X[:, 0], X[:, 1], '.')
ax.plot(centres[:, 0], centres[:, 1], 's')
plt.show()

It is quite clear that they do **not** represent the arithmetic mean of any of the data. 
The second step (the first of the E-M loop) is to assign all of the points to their nearest cluster centre. 
This can be achieved very efficiently by using NumPy and array broadcasting. 
First, we compute the vector from each of the data, `X`, to each of the current cluster centres. 

In [ ]:
vectors = X[:, np.newaxis] - centres
vectors.shape

Notice that the shape of this array is `(number_of_data, number_of_clusters, dimensions)`. 
The magnitude of the vector can describe the distance from the data to the current centres, we want the magnitude along the dimensions axis of the array (the final axis, hence `-1`). 

In [ ]:
magnitudes = np.linalg.norm(vectors, axis=-1)
magnitudes.shape

We now have the magnitude of the vector from each data point to each of the cluster centres. 
We want to know which centre each datapoint is closest to, for this, we can use the `argmin()` method of the NumPy array. 
This returns the index of the minimum value (the method `min()` will return the actual value, but we aren't actually interested in this).
The `argmin()` method should only be run on the number of clusters axis so that we end up with an array of 300 indices. 

In [ ]:
labels = magnitudes.argmin(axis=1)
print(labels)

This array, `labels`, is the assignment of each of the data points to the closest cluster centres, completing the E-step in the E-M loop. 
Now, we need to update the cluster centres, with the new arithmetic mean of the data that are assigned to it. 
The data can be split into groups using logical slicing, i.e., to get all of the values with the label `0`.

In [ ]:
X[labels == 0]

Above, are all of the datapoints in associated with the first cluster centre, and the mean (along the datapoint axis can be found as).

In [ ]:
X[labels == 0].mean(axis=0)

And it is possible to perform this for all four of the clusters with simple list comprehension. 

In [10]:
new_centres = np.array([X[labels == i].mean(axis=0) for i in range(n_clusters)])

The new centres can be visualised as shown below (this time with triangles). 

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(X[:, 0], X[:, 1], '.')
ax.plot(centres[:, 0], centres[:, 1], 's')
ax.plot(new_centres[:, 0], new_centres[:, 1], '^')
plt.show()

Finally, to update the centres, we overwrite the `centres` object. 

In [17]:
centres = new_centres

This process should be completed until the centres do not change meaning that we can use an iterative approach. 
See the code cell below, which loops over this process until the is no change in the centres.

In [22]:
from sklearn.datasets import make_blobs
import numpy as np

X, y_true = make_blobs(n_samples=300, centers=4, cluster_std=1, random_state=1)
n_clusters = 4
i = np.random.randint(0, X.shape[0], size=n_clusters)
centres = X[i]
diff = np.inf
while np.sum(diff) > 0.00000001:
    vectors = X[:, np.newaxis] - centres
    magnitudes = np.linalg.norm(vectors, axis=-1)
    labels = magnitudes.argmin(axis=1)
    new_centres = np.array([X[labels == i].mean(axis=0) for i in range(n_clusters)])
    diff = np.abs(centres - new_centres)
    centres = new_centres

We show the results below, with the different clusters identified by colour and the cluster centres marked with a black square. 

In [ ]:
from sklearn.cluster import KMeans

fig, ax = plt.subplots(figsize=(6, 4))
for i in range(n_clusters):
    ax.plot(X[labels == i][:, 0], X[labels == i][:, 1], '.')
ax.plot(centres[:, 0], centres[:, 1], 'ks')
plt.show()


There are some important issues that one should be aware of in using the simple E-M algorithm discussed above: 
- The globally optimal result may never be achieved. As is the case with all optimisation routines, although the result is improving, it may not be moving to the globally optimal solution. 
- *k*-means is limited to linear cluster boundaries. The fact that *k*-means is finding samples as close as possible in cartesian space means that the clustering cannot have more complex geometries. 
- *k*-means can be slow for a large number of samples. Each iteration must access every point in the dataset (and in our implementation, it accesses each point `n_clusters` number of times)!
- The number of clusters must be selected beforehand. We must have some {term}`a priori` knowledge about our dataset to effectively apply a *k*-means clustering.

This final point is a common concern in *k*-means clustering, and other clustering algorithms. 
Therefore, let's look at a popular tool used to find the "correct" number of clusters. 